In [3]:
import puffin.tardis_importer as ti
import numpy as np
import pandas as pd
coins = ['UNI', 'USDT', 'XRP', 'YFI']
for symbol in coins:
    spot_trades = ti.generate_trade_data_from_gzip(f'{symbol}/USD', f'./datasets/ftx_trades_2021-10-14_{symbol}-USD.csv.gz')
    for day in range(15, 29):
        t = ti.generate_trade_data_from_gzip(f'{symbol}/USD', f'./datasets/ftx_trades_2021-10-{day}_{symbol}-USD.csv.gz')
        spot_trades = spot_trades.append(t)
    spot_trades['timestamp'] = spot_trades['exchange_timestamp_nanos'] // 1000000000
    spot_trades['timestamp_shifted'] = (spot_trades['exchange_timestamp_nanos'] + 300000000) // 1000000000
    def vwap(df):
        return np.average(df['price'], weights=df['size'])
    data = pd.DataFrame(index=range(spot_trades['timestamp'].min(), spot_trades['timestamp'].max() + 1))
    original = spot_trades[['timestamp', 'price', 'size']]
    original = original.groupby('timestamp').apply(vwap).to_frame('vwap_original')
    shifted = spot_trades[['timestamp_shifted', 'price', 'size']]
    shifted = shifted.groupby('timestamp_shifted').apply(vwap).to_frame('vwap_shifted')
    data = pd.merge(left=data, right=original, how='left', left_index=True, right_index=True)
    data = pd.merge(left=data, right=shifted, how='left', left_index=True, right_index=True)
    data = data.fillna(method='ffill')
    data = data.diff()
    data = data.dropna(how='any')
    autocorr_original = data['vwap_original'].rolling(60).apply(lambda x: x.autocorr(), raw=False).to_frame()
    autocorr_shifted = data['vwap_shifted'].rolling(60).apply(lambda x: x.autocorr(), raw=False).to_frame()
    autocorr_data = pd.merge(left=autocorr_original, right=autocorr_shifted, how='left', left_index=True, right_index=True)
    threshold = -0.15
    print([symbol, len(autocorr_data[autocorr_data.vwap_shifted < threshold])/len(autocorr_data), len(autocorr_data[np.logical_and(autocorr_data.vwap_shifted < threshold, autocorr_data.vwap_original < 0)])/len(autocorr_data[autocorr_data.vwap_shifted < threshold])])

['UNI', 0.02342133244032784, 0.8927653686499308]
['USDT', 0.3692804733417337, 0.8889107127558877]
['XRP', 0.05225442250147764, 0.87039470769776]
['YFI', 0.018532421707115516, 0.8971604629860938]


In [2]:
symbol

'TRYB'